The structure of the files is as follows:


number of customers
number of available depots

coordinates for the depots (x and y)

coordinates for the customers

vehicle capacity

depot capacities (for Tuzun instances, each one is equal to the total demand as there is no capacity on the depots)

customers demands

opening costs for the depots

opening cost of a route (cost of a vehicle)

0 or 1 (0 means that the costs are integer - 1 that costs are real)


To calculate the matrix distance (or the cost to link any 2 points A and B in the graph), we use the mathematical formula:

sqrt( (xA-xB)² + (yA-yB)² )

The results are stored in a float variable (in C language) if the costs are real (code 1)
The result is multiplied by 100 and truncked to be stored in an integer variable if the costs are interger (code 0).

Example of a file:

20
5

6	7
19	44
37	23
35	6
5	8

20	35
8	31
29	43
18	39
19	47
31	24
38	50
33	21
2	27
1	12
26	20
20	33
15	46
20	26
17	19
15	12
5	30
13	40
38	5
9	40

70

140
140
140
140
140

17
18
13
19
12
18
13
13
17
20
16
18
15
11
18
16
15
15
15
16

10841
11961
6091
7570
7497

1000

0



In [1]:
from docplex.mp.model import Model
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import time

Parser

In [2]:
def parse_file(file_path):
    with open(file_path, 'r') as file:
        lines = [line.strip() for line in file if line.strip()]

    num_customers, num_depots = int(lines[0]), int(lines[1])
    depots = [tuple(map(float, lines[i].split())) for i in range(2, 2 + num_depots)]
    customers = [tuple(map(float, lines[i].split())) for i in range(2 + num_depots, 2 + num_depots + num_customers)]
    vehicle_capacity = float(lines[2 + num_depots + num_customers])
    depot_capacities = list(map(float, lines[3 + num_depots + num_customers:3 + 2 * num_depots + num_customers]))
    customer_demands = list(
        map(float, lines[3 + 2 * num_depots + num_customers:3 + 2 * num_depots + 2 * num_customers]))
    depot_opening_costs = list(
        map(float, lines[3 + 2 * num_depots + 2 * num_customers:3 + 3 * num_depots + 2 * num_customers]))
    route_opening_cost = float(lines[3 + 3 * num_depots + 2 * num_customers])

    all_points = depots + customers
    distance_matrix = [
        [math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2) for x2, y2 in all_points] for x1, y1 in all_points
    ]

    return {
        "num_customers": num_customers,
        "num_depots": num_depots,
        "depots": depots,
        "customers": customers,
        "vehicle_capacity": vehicle_capacity,
        "depot_capacities": depot_capacities,
        "customer_demands": customer_demands,
        "depot_opening_costs": depot_opening_costs,
        "route_opening_cost": route_opening_cost,
        "distance_matrix": distance_matrix,
    }

In [3]:
def modelo_scf(parsed_data):

    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customers_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    costs = parsed_data["distance_matrix"]


    mdl = Model(name="SCF")

    # Variables
    nodos = list(range(n + m))
    x = mdl.binary_var_matrix(nodos, nodos, name="x")  # Ruta entre nodos
    f = mdl.continuous_var_matrix(nodos, nodos, name="f")  # Flujo
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in nodos for j in nodos if i != j) +
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m))
    )

    # Restricciones
    # 1. Cada cliente es atendido exactamente una vez
    for i in range(m, m + n):
        mdl.add_constraint(mdl.sum(x[i, j] for j in nodos if j != i) == 1)
        mdl.add_constraint(mdl.sum(x[j, i] for j in nodos if j != i) == 1)

    # 2. Cada ruta debe comenzar y terminar en un depósito
    for d in range(m):
        mdl.add_constraint(mdl.sum(x[d, j] for j in range(m, m + n)) <= y[d])
        mdl.add_constraint(mdl.sum(x[j, d] for j in range(m, m + n)) <= y[d])

    # 3. Restricciones de flujo
    for i in range(m, m + n):
        mdl.add_constraint(
            mdl.sum(f[i, j] for j in nodos if j != i) -
            mdl.sum(f[j, i] for j in nodos if j != i) == customers_demands[i - m]
        )
    for i, j in [(i, j) for i in nodos for j in nodos if i != j]:
        mdl.add_constraint(f[i, j] <= vehicle_capacity * x[i, j])

    # 4. Capacidades de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(f[d, j] for j in range(m, m + n)) <= depot_capacities[d]
        )

    return mdl


In [4]:
def modelo_cda(parsed_data):
    n = parsed_data["num_customers"]
    m = parsed_data["num_depots"]
    depots = parsed_data["depots"]
    customers = parsed_data["customers"]
    vehicle_capacity = parsed_data["vehicle_capacity"]
    depot_capacities = parsed_data["depot_capacities"]
    customer_demands = parsed_data["customer_demands"]
    opening_costs_depots = parsed_data["depot_opening_costs"]
    costs = parsed_data["distance_matrix"]

    mdl = Model(name="CDA")

    # Variables
    v = mdl.binary_var_matrix(n, m, name="v")  # Cliente asignado a depósito
    x = mdl.binary_var_matrix(n + m, n + m, name="x")  # Uso de arcos
    y = mdl.binary_var_list(m, name="y")  # Uso de depósitos

    # Función objetivo
    mdl.minimize(
        mdl.sum(costs[i][j] * x[i, j] for i in range(n + m) for j in range(n + m)) +
        mdl.sum(opening_costs_depots[d] * y[d] for d in range(m))
    )

    # Restricciones
    # 1. Cada cliente asignado a un único depósito
    for i in range(n):
        mdl.add_constraint(mdl.sum(v[i, d] for d in range(m)) == 1)

    # 2. Si un cliente está asignado a un depósito, el depósito debe estar activo
    for i in range(n):
        for d in range(m):
            mdl.add_constraint(v[i, d] <= y[d])

    # 3. Relación entre asignaciones y rutas
    for i in range(n):
        for j in range(n):
            for d in range(m):
                mdl.add_constraint(v[i, d] + x[i + m, j + m] <= v[j, d] + 1)

    # 4. Capacidades de los depósitos
    for d in range(m):
        mdl.add_constraint(
            mdl.sum(customer_demands[i] * v[i, d] for i in range(n)) <= depot_capacities[d]
        )

    # 5. Capacidad del vehículo
    for i, j in [(i, j) for i in range(n + m) for j in range(n + m) if i != j]:
        mdl.add_constraint(x[i, j] <= vehicle_capacity)

    return mdl


In [5]:
file_path = 'Instances/Benchmark_1/coord20-5-1.dat'

parsed_data = parse_file(file_path)

In [6]:
mdl_scf = modelo_scf(parsed_data)
mdl_scf.print_information()

Model: SCF
 - number of variables: 1255
   - binary=630, integer=0, continuous=625
 - number of constraints: 675
   - linear=675
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [7]:
mdl_cda = modelo_cda(parsed_data)
mdl_cda.print_information()

Model: CDA
 - number of variables: 730
   - binary=730, integer=0, continuous=0
 - number of constraints: 2725
   - linear=2725
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


In [8]:
def solve_instance(model):
    start_time = time.time()
    solution = model.solve()
    end_time = time.time()

    metrics = {
        "Modelo": model.name,
        "Instancia": file_path,
        "Número de Variables": model.number_of_variables,
        "Número de Restricciones": model.number_of_constraints,
        "Valor Función Objetivo": model.objective_value if solution is not None else "N/A",
        "Tiempo de Cómputo (s)": end_time - start_time
    }
    return metrics

In [9]:
metrics_scf = solve_instance(model=mdl_scf)
metrics_scf

TypeError: solve_instance() got an unexpected keyword argument 'model_name'

In [10]:
metrics_cda = solve_instance(model=mdl_cda)
metrics_cda

{'Modelo': 'CDA',
 'Instancia': 'Instances/Benchmark_1/coord20-5-1.dat',
 'Número de Variables': 730,
 'Número de Restricciones': 2725,
 'Valor Función Objetivo': 21158.0,
 'Tiempo de Cómputo (s)': 0.1500720977783203}